In [49]:
import os
import cv2

In [50]:
# Initialize the YuNet model
yunet_model_path = os.path.join('yunet', 'models', 'face_detection_yunet_2023mar.onnx')
yunet = cv2.FaceDetectorYN.create(
    model=      yunet_model_path,
    config=     "",
    input_size= (0, 0)
)

In [51]:
def read_faceboxes(annotations_path: str) -> list[dict[str, any]]:
    """
    Read faceboxes information from an annotations file.

    Parameters:
    - annotations_path (str): Path to the annotations file.

    Returns:
    - List[Dict[str, any]]: List of dictionaries representing faceboxes.
    """
    faceboxes = []
    num_lines = 0

    with open(annotations_path, 'r') as file:
        # Loop through the lines in the annotation file
        for i, line in enumerate(file):
            num_lines = i + 1
            # Split the line into values
            values = line.strip().split()

            # Check for the end of the annotations file
            if len(values) == 0:
                break

            # Get the image path and number of faces in the image
            img_path = values[0]
            face_index = 1

            # Error check the line to ensure it has the expected format
            try:
                num_faces = int(values[face_index])
            except ValueError:
                print(f'Line {i}: Warning: Skipping Line; Incorrect Format:')
                print(f'- Line: "{line.strip()}"')
                continue

            # Loop through every face in the picture
            for _ in range(num_faces):
                # Get the variables for face 'j' in the picture
                face_x = int(values[face_index + 1])
                face_y = int(values[face_index + 2])
                face_w = int(values[face_index + 3])
                face_h = int(values[face_index + 4])
                face_index += 4

                # Save the variables as a facebox
                facebox = {
                    'filename': img_path,
                    'x': face_x,
                    'y': face_y,
                    'w': face_w,
                    'h': face_h
                }
                faceboxes.append(facebox)

    # Print the result for debugging
    print(f'Info: [read_faceboxes] '
          f'Found {len(faceboxes)} face{"s" if len(faceboxes) > 1 else ""} '
          f'in {num_lines} picture{"s" if num_lines > 1 else ""}')
    return faceboxes

In [52]:
faceboxes = read_faceboxes(os.path.join('evaluation', 'info.dat'))

Info: [read_faceboxes] Found 17 faces in 1 picture
